In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import *
import pandas as pd
import numpy as np
from glob import glob
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
%matplotlib inline

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
data_dir = os.path.join(os.pardir, 'data', 'whale', 'imgs')
batch_size=32
num_class = 447
img_shape=(3, 300, 200)
target_size = (300,200)

In [3]:
data_dir

'../data/whale/imgs'

In [4]:
def get_tr_batches(dr = data_dir+ '/train', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=True, target_size=target_size, batch_size=64)

In [5]:
def get_val_batches(dr = data_dir+ '/valid', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=False, target_size=target_size, batch_size=batch_size)

In [6]:
def fit_model(model, epochs, tr_b = get_tr_batches(), val_b = get_val_batches()):
    model.fit_generator(tr_b, tr_b.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, 
                    nb_epoch=epochs)

Found 3644 images belonging to 447 classes.
Found 900 images belonging to 447 classes.


## Linear Model

In [19]:
def get_lm():
    return Sequential([BatchNormalization(axis=1, input_shape=img_shape),
                       Flatten(),
                       Dense(num_class, activation='softmax')
                      ])

In [20]:
tr_b = get_tr_batches()
val_b = get_val_batches()

Found 3644 images belonging to 447 classes.
Found 900 images belonging to 447 classes.


In [21]:
model = get_lm()

In [22]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.fit_generator?

In [ ]:
model.fit_generator(tr_b, tr_b.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, 
                    nb_epoch=1)

In [48]:
model.optimizer.lr = 0.001

In [49]:
model.fit_generator(tr_b, tr_b.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, 
                    nb_epoch=1)

Epoch 1/1
3644/3644 [==============================] - 47s - loss: 10.1541 - acc: 0.0749 - val_loss: 12.0137 - val_acc: 0.0000e+00


In [50]:
model.optimizer.lr = 0.01

In [51]:
model.fit_generator(tr_b, tr_b.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, 
                    nb_epoch=6)

Epoch 1/6
3644/3644 [==============================] - 47s - loss: 9.6548 - acc: 0.0771 - val_loss: 11.5356 - val_acc: 0.0033
Epoch 2/6
3644/3644 [==============================] - 41s - loss: 9.3997 - acc: 0.0686 - val_loss: 11.1526 - val_acc: 0.0067
Epoch 3/6
3644/3644 [==============================] - 41s - loss: 9.1618 - acc: 0.0741 - val_loss: 10.7962 - val_acc: 0.0100
Epoch 4/6
3644/3644 [==============================] - 43s - loss: 8.9295 - acc: 0.0757 - val_loss: 10.9093 - val_acc: 0.0044
Epoch 5/6
3644/3644 [==============================] - 42s - loss: 8.7766 - acc: 0.0714 - val_loss: 10.9804 - val_acc: 0.0022
Epoch 6/6
3644/3644 [==============================] - 41s - loss: 8.4072 - acc: 0.0897 - val_loss: 11.0688 - val_acc: 0.0033


In [47]:
model.fit_generator(tr_b, tr_b.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, 
                    nb_epoch=1)

Epoch 1/1
3644/3644 [==============================] - 47s - loss: 11.4567 - acc: 0.0080 - val_loss: 12.6913 - val_acc: 0.0022


# One layer Net

In [62]:
Dense?

In [27]:
def get_net():
    return Sequential([BatchNormalization(axis=1, input_shape=img_shape),
                       Flatten(),
                       Dense(256, activation='relu'),
                       Dense(num_class, activation='softmax')
                      ])


In [28]:
model = get_net()

In [29]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
fit_model(model,1)


Epoch 1/1
3644/3644 [==============================] - 51s - loss: 11.7812 - acc: 0.0080 - val_loss: 15.6901 - val_acc: 0.0056


In [81]:
model.optimizer.lr = 0.001
fit_model(model,4)


Epoch 1/4
3644/3644 [==============================] - 48s - loss: 8.6638 - acc: 0.0999 - val_loss: 14.3431 - val_acc: 0.0033
Epoch 2/4
3644/3644 [==============================] - 41s - loss: 8.0003 - acc: 0.1685 - val_loss: 12.9595 - val_acc: 0.0033
Epoch 3/4
3068/3644 [========================>.....] - ETA: 4s - loss: 7.5326 - acc: 0.2122

KeyboardInterrupt: 

# Simple Convnet

In [36]:
MaxPooling2D?

In [34]:
from keras.layers import ZeroPadding2D

In [38]:
def get_net():
    return Sequential([BatchNormalization(axis=1, input_shape=img_shape),
                       Conv2D(64, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Conv2D(32, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Flatten(),
                       Dense(256, activation='relu'),
                       Dense(num_class, activation='softmax')
                      ])



In [39]:
model = get_net()

In [40]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
fit_model(model,1)


Epoch 1/1
3644/3644 [==============================] - 53s - loss: 6.1009 - acc: 0.0066 - val_loss: 6.3322 - val_acc: 0.0067


In [42]:
model.optimizer.lr = 0.001

In [43]:
fit_model(model,2)


Epoch 1/2
3644/3644 [==============================] - 48s - loss: 5.8812 - acc: 0.0247 - val_loss: 6.4452 - val_acc: 0.0044
Epoch 2/2
3644/3644 [==============================] - 46s - loss: 5.4817 - acc: 0.0639 - val_loss: 6.5381 - val_acc: 0.0089


In [44]:
model.optimizer.lr = 0.0001

In [45]:
fit_model(model,1)


Epoch 1/1
3644/3644 [==============================] - 48s - loss: 4.5977 - acc: 0.1998 - val_loss: 6.5487 - val_acc: 0.0056


# Convnet with Dropout

In [54]:
def get_drop_net():
    return Sequential([BatchNormalization(axis=1, input_shape=img_shape),
                       Conv2D(64, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Dropout(0.9),
                       Conv2D(32, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Dropout(0.9),
                       Flatten(),
                       Dense(256, activation='relu'),
                       Dropout(0.9),
                       Dense(num_class, activation='softmax')
                      ])



In [55]:
model = get_drop_net()

In [56]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [57]:
fit_model(model,1)


Epoch 1/1
3644/3644 [==============================] - 50s - loss: 8.4548 - acc: 0.0071 - val_loss: 6.1017 - val_acc: 0.0067


In [58]:
model.optimizer.lr = 0.001

In [59]:
fit_model(model,1)


Epoch 1/1
3644/3644 [==============================] - 50s - loss: 6.1149 - acc: 0.0104 - val_loss: 6.1008 - val_acc: 0.0067


In [60]:
model.optimizer.lr = 0.00001

In [61]:
fit_model(model,1)

Epoch 1/1
3644/3644 [==============================] - 51s - loss: 6.1150 - acc: 0.0107 - val_loss: 6.0999 - val_acc: 0.0067


# Convnet more params

In [7]:
def get_drop_net():
    return Sequential([BatchNormalization(axis=1, input_shape=img_shape),
                       Conv2D(128, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Conv2D(64, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Flatten(),
                       Dense(512, activation='relu'),
                       Dropout(0.9),
                       Dense(256, activation='relu'),
                       Dropout(0.9),
                       Dense(num_class, activation='softmax')
                      ])



In [8]:
model = get_drop_net()

In [9]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
fit_model(model,1)


Epoch 1/1
3644/3644 [==============================] - 81s - loss: 7.0686 - acc: 0.0027 - val_loss: 6.1019 - val_acc: 0.0022


In [11]:
model.optimizer.lr = 0.001

In [12]:
fit_model(model,1)


Epoch 1/1
3644/3644 [==============================] - 79s - loss: 6.1008 - acc: 0.0060 - val_loss: 6.1008 - val_acc: 0.0000e+00


# Predict on Test Set

In [13]:
def get_test_batches(dr = data_dir+ '/test', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=False, target_size=target_size, batch_size=batch_size)

In [14]:
test_batches = get_test_batches()

Found 6925 images belonging to 1 classes.


In [27]:
model.predict_generator?

In [15]:
preds = model.predict_generator(test_batches, val_samples=test_batches.nb_sample)

In [16]:
preds

array([[ 0.0022,  0.0023,  0.0022, ...,  0.0022,  0.0022,  0.0022],
       [ 0.0022,  0.0023,  0.0022, ...,  0.0022,  0.0022,  0.0022],
       [ 0.0022,  0.0024,  0.0022, ...,  0.0022,  0.0023,  0.0022],
       ..., 
       [ 0.0022,  0.0023,  0.0022, ...,  0.0022,  0.0022,  0.0022],
       [ 0.0022,  0.0023,  0.0022, ...,  0.0022,  0.0022,  0.0022],
       [ 0.0022,  0.0022,  0.0022, ...,  0.0022,  0.0022,  0.0022]], dtype=float32)

In [17]:
whale_dir = os.path.join(os.pardir, 'data', 'whale')

In [18]:
samp = pd.read_csv(whale_dir+'/sample_submission.csv')

In [19]:
tr_batches = get_tr_batches()

Found 3644 images belonging to 447 classes.


In [23]:
def create_sub(preds, test_batches, train_batches):
    file_names = np.array([f[f.find('/') +1:] for f in test_batches.filenames])
    
    idx_class = [(v, k) for k,v in tr_batches.class_indices.items()]
    idx_class= sorted(idx_class, key=lambda x: x[0])
    classes = [x[1] for x in idx_class]
    data = np.hstack((file_names[:, np.newaxis], preds))
    columns = ['Image'] + classes
    return pd.DataFrame(data, columns=columns)

In [24]:
sub = create_sub(preds, test_batches, tr_batches)

In [27]:
sub.head()

,Image,whale_00195,whale_00442,whale_02411,whale_02608,whale_02839,whale_03103,whale_03227,whale_03623,whale_03728,...,whale_98618,whale_98633,whale_98645,whale_98746,whale_98939,whale_98996,whale_99243,whale_99326,whale_99558,whale_99573
0,w_8037.jpg,0.00224931770936,0.00225193682127,0.0022017066367,0.00221168785356,0.00220592762344,0.00219153170474,0.00228456570767,0.00223549129441,0.00225559389219,...,0.00223504728638,0.00222577597015,0.0022574160248,0.00227861781605,0.00222936272621,0.00220952951349,0.00222373357974,0.00221352605149,0.00223460420966,0.00222191866487
1,w_4494.jpg,0.00223772507161,0.00229111709632,0.00221004500054,0.00220582145266,0.00217056111433,0.00221282010898,0.00230586202815,0.00227250088938,0.00226316298358,...,0.00227137259208,0.00223008403555,0.00221053534187,0.00232449523173,0.00222057430074,0.00217467756011,0.00221117213368,0.0021644087974,0.00224630045705,0.00220335205086
2,w_4673.jpg,0.00222146860324,0.00235310429707,0.00215468043461,0.00221396819688,0.00219106394798,0.00213481672108,0.00229335622862,0.00220231269486,0.00229474925436,...,0.00223404099233,0.0021764645353,0.00225853268057,0.0023292817641,0.00228868843988,0.00219468260184,0.00220470014028,0.00219992036,0.0022615713533,0.00217787874863
3,w_8273.jpg,0.00224115955643,0.00226141489111,0.00222268467769,0.00221747695468,0.00219283136539,0.00222086696886,0.00227946415544,0.0022596239578,0.00224920781329,...,0.00225545209832,0.00223349686712,0.00222505396232,0.0022899643518,0.00222731102258,0.00220002117567,0.00222342298366,0.00219295267016,0.00224516098388,0.00221747206524
4,w_7611.jpg,0.00220042630099,0.00232659908943,0.00213251449168,0.00223747314885,0.0022390789818,0.00213853619061,0.00232293806039,0.00220596184954,0.00229228730313,...,0.00223859306425,0.00212115887552,0.00235242280178,0.00231035868637,0.00224314234219,0.00218514748849,0.00223006005399,0.00221631769091,0.00231484137475,0.00222031935118


In [29]:
sub.shape

(6925, 448)

In [30]:
sub.to_csv('base.csv', index=False)

In [32]:
from IPython.display import FileLink

FileLink('base.csv')

/home/ubuntu/courses/deeplearning1/nbs/whale/base.csv

Scores: 6.10062

## Analysis

In [33]:
sub = sub.iloc[:, 1:]

In [36]:
sub.head()

,whale_00195,whale_00442,whale_02411,whale_02608,whale_02839,whale_03103,whale_03227,whale_03623,whale_03728,whale_03935,...,whale_98618,whale_98633,whale_98645,whale_98746,whale_98939,whale_98996,whale_99243,whale_99326,whale_99558,whale_99573
0,0.00224931770936,0.00225193682127,0.0022017066367,0.00221168785356,0.00220592762344,0.00219153170474,0.00228456570767,0.00223549129441,0.00225559389219,0.00226420070976,...,0.00223504728638,0.00222577597015,0.0022574160248,0.00227861781605,0.00222936272621,0.00220952951349,0.00222373357974,0.00221352605149,0.00223460420966,0.00222191866487
1,0.00223772507161,0.00229111709632,0.00221004500054,0.00220582145266,0.00217056111433,0.00221282010898,0.00230586202815,0.00227250088938,0.00226316298358,0.00229386449791,...,0.00227137259208,0.00223008403555,0.00221053534187,0.00232449523173,0.00222057430074,0.00217467756011,0.00221117213368,0.0021644087974,0.00224630045705,0.00220335205086
2,0.00222146860324,0.00235310429707,0.00215468043461,0.00221396819688,0.00219106394798,0.00213481672108,0.00229335622862,0.00220231269486,0.00229474925436,0.00227660406381,...,0.00223404099233,0.0021764645353,0.00225853268057,0.0023292817641,0.00228868843988,0.00219468260184,0.00220470014028,0.00219992036,0.0022615713533,0.00217787874863
3,0.00224115955643,0.00226141489111,0.00222268467769,0.00221747695468,0.00219283136539,0.00222086696886,0.00227946415544,0.0022596239578,0.00224920781329,0.00226887804456,...,0.00225545209832,0.00223349686712,0.00222505396232,0.0022899643518,0.00222731102258,0.00220002117567,0.00222342298366,0.00219295267016,0.00224516098388,0.00221747206524
4,0.00220042630099,0.00232659908943,0.00213251449168,0.00223747314885,0.0022390789818,0.00213853619061,0.00232293806039,0.00220596184954,0.00229228730313,0.00233325292356,...,0.00223859306425,0.00212115887552,0.00235242280178,0.00231035868637,0.00224314234219,0.00218514748849,0.00223006005399,0.00221631769091,0.00231484137475,0.00222031935118


In [43]:
maxes = sub.max(axis=1)

In [44]:
maxes.max()

0.0025273603387200002

In [45]:
maxes.min()

0.00225573289208

In [48]:
maxes.min() * 447

1.00831260275976

In [51]:
print(sub.iloc[1,:].min())
sub.iloc[1,:].max()

0.00212844577618


'0.00235452363268'

Our model is giving equal probability to each whale. How might we prevent that from happening? Why is that happening?

- It doesn't have enough information to identify the whale. We could increase the resolution size of the images. We could crop the images so we just have the whale, then crop the images. Since we have aerial photos, the orientation of the images doesn't matter. You could rotate them in a 360 degree range to increase the amount of data you have.
- That's a really simple thing to do. Maybe you try that.
- Could it be the way you've set up your train and validation sets? How did you set it up? You made sure that your training set had at least one example for each whale, then you remainder into train and validation sets. Could this mean that rare whales are disproportionately common in your training set? That could be.

- Maybe eventually you should create a lot more training images by creating a bunch of random rotations of each image. Then you divide the images in a way so that the training and validation sets would be more representative of the overall distribution of whales. Do that with the resized whales. Wouldn't want to do that on the massive original whale images`

- You could also use a more powerful model. Try more filters or hidden units. Try pretrained vgg features. That would probably be the simplest thing to do.




## Rotate Images

In [52]:
def get_drop_net():
    return Sequential([BatchNormalization(axis=1, input_shape=img_shape),
                       Conv2D(128, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Conv2D(64, 3,3, activation='relu'),
                       MaxPooling2D((2,2),strides=(2,2)),
                       Flatten(),
                       Dense(512, activation='relu'),
                       Dropout(0.9),
                       Dense(256, activation='relu'),
                       Dropout(0.9),
                       Dense(num_class, activation='softmax')
                      ])



In [53]:
model = get_drop_net()

In [54]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [56]:
ImageDataGenerator?

In [57]:
tr_b  = get_tr_batches(gen=ImageDataGenerator(rotation_range=180))
val_b = get_val_batches()

Found 3644 images belonging to 447 classes.
Found 900 images belonging to 447 classes.


In [60]:
fit_model(model, epochs=1, tr_b=tr_b)

Epoch 1/1
3644/3644 [==============================] - 77s - loss: 7.8723 - acc: 0.0030 - val_loss: 6.1024 - val_acc: 0.0022


In [61]:
model.optimizer.lr = 0.001

In [62]:
fit_model(model, epochs=1, tr_b=tr_b)

Epoch 1/1
3644/3644 [==============================] - 77s - loss: 6.1027 - acc: 0.0063 - val_loss: 6.1008 - val_acc: 0.0056


In [63]:
fit_model(model, epochs=10, tr_b=tr_b)

Epoch 1/10
3644/3644 [==============================] - 77s - loss: 6.1000 - acc: 0.0085 - val_loss: 6.0998 - val_acc: 0.0111
Epoch 2/10
3644/3644 [==============================] - 77s - loss: 6.0997 - acc: 0.0049 - val_loss: 6.0989 - val_acc: 0.0089
Epoch 3/10
3644/3644 [==============================] - 77s - loss: 6.0981 - acc: 0.0058 - val_loss: 6.0978 - val_acc: 0.0078
Epoch 4/10
3644/3644 [==============================] - 78s - loss: 6.0969 - acc: 0.0063 - val_loss: 6.0969 - val_acc: 0.0100
Epoch 5/10
3644/3644 [==============================] - 77s - loss: 6.0968 - acc: 0.0080 - val_loss: 6.0960 - val_acc: 0.0100
Epoch 6/10
3644/3644 [==============================] - 77s - loss: 6.0949 - acc: 0.0085 - val_loss: 6.0950 - val_acc: 0.0100
Epoch 7/10
3580/3644 [============================>.] - ETA: 1s - loss: 6.0937 - acc: 0.0061

KeyboardInterrupt: 